In [17]:
import pandas as pd
import numpy as np
from typing import Dict, List
import math

In [18]:
def parse_timestamp_to_utc(timestamp):
    """밀리초 단위 유닉스 타임스탬프를 UTC datetime으로 변환"""
    try:
        # 밀리초를 초로 변환
        ts_seconds = float(timestamp) / 1000
        return datetime.fromtimestamp(ts_seconds, tz=timezone.utc)
    except (ValueError, TypeError):
        raise ValueError(f"Invalid timestamp format: {timestamp}")

In [19]:
data = pd.read_csv(r"C:\Dev\TMBearing\testdata.csv")  

# UTC 시간 및 초 단위 컬럼 추가
data['time'] = data['timestamp'].apply(parse_timestamp_to_utc)
data['second'] = (data['timestamp'].apply(float) / 1000).astype(int)  # 초 단위로 내림

In [20]:
data

,magic,deviceID,commandId,timestamp,ch1,ch2,ch3,time,second
0,aaaa,2732,10,1734381207178,-0.001391,-0.002563,-1.025016,2024-12-16 20:33:27.178000+00:00,1734381207
1,aaaa,2732,10,1734381207179,-0.001406,-0.002703,-1.025359,2024-12-16 20:33:27.179000+00:00,1734381207
2,aaaa,2732,10,1734381207180,-0.001172,-0.003031,-1.025375,2024-12-16 20:33:27.180000+00:00,1734381207
3,aaaa,2732,10,1734381207181,-0.001156,-0.003172,-1.025375,2024-12-16 20:33:27.181000+00:00,1734381207
4,aaaa,2732,10,1734381207182,-0.001203,-0.003016,-1.026000,2024-12-16 20:33:27.182000+00:00,1734381207
...,...,...,...,...,...,...,...,...,...
44578375,aaaa,2732,10,1734427292363,-0.000750,-0.002406,-1.027016,2024-12-17 09:21:32.363000+00:00,1734427292
44578376,aaaa,2732,10,1734427292364,-0.000797,-0.002422,-1.025516,2024-12-17 09:21:32.364000+00:00,1734427292
44578377,aaaa,2732,10,1734427292365,-0.000094,-0.002797,-1.025938,2024-12-17 09:21:32.365000+00:00,1734427292
44578378,aaaa,2732,10,1734427292366,-0.000266,-0.002688,-1.026109,2024-12-17 09:21:32.366000+00:00,1734427292


In [22]:
def compute_channel_stats(data: pd.DataFrame) -> Dict[str, pd.DataFrame]:
    # 그룹화 및 통계 계산
    grouped = data.groupby("second")
    
    def stats_func(series: pd.Series) -> Dict[str, float]:
        return {
            "mean": series.mean(),
            "peak": np.max(np.abs([series.min(), series.max()])),
            "max": series.max(),
            "min": series.min(),
            "rms": np.sqrt(np.mean(series ** 2))
        }
    
    # 병렬 처리를 위해 각 채널별로 분리
    results = {}
    for channel in ["ch1", "ch2", "ch3"]:
        stats_df = grouped.agg({channel: stats_func}).rename(columns={channel: channel})
        stats_df.columns = [f"{channel}_{stat}" for stat in stats_df.columns]
        stats_df = stats_df.reset_index()
        results[channel] = stats_df
    
    return results

In [23]:
channel_stats = compute_channel_stats(data)

In [26]:
channel_stats['ch1']

,second,ch1_ch1
0,1734381207,"{'mean': -0.0012317328205614334, 'peak': 0.002..."
1,1734381208,"{'mean': -0.001229640629477167, 'peak': 0.0021..."
2,1734381209,"{'mean': -0.0012428125019359366, 'peak': 0.002..."
3,1734381210,"{'mean': -0.0012212656279006714, 'peak': 0.002..."
4,1734381211,"{'mean': -0.0012256875022739653, 'peak': 0.007..."
...,...,...
44587,1734427288,"{'mean': -0.0003340625007949361, 'peak': 0.001..."
44588,1734427289,"{'mean': -0.000331984376280126, 'peak': 0.0012..."
44589,1734427290,"{'mean': -0.0002994062512752524, 'peak': 0.001..."
44590,1734427291,"{'mean': -0.0003360625007062114, 'peak': 0.001..."
